In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np

## Config

In [ ]:
SPREADSHEET_KEY = os.getenv("SPREADSHEET_KEY")
TARGET_END_DATE = pd.to_datetime("2021-09-19")
FILE_PATH = "dados/Superloja_original.xls"
TODAY = datetime.today().date()

## Preprocessing functions

In [ ]:
def shift_dates_to_end_at(df, target_end_date):
    """
    Faz com que Data da compra (e Data de envio) sejam deslocados no tempo
    para que a maior Data da compra ocorra no dia target_end_date
    """
    df = df.copy()
    curr_end_date = df["Data da compra"].max()

    print(f"\tDeslocando data de compra {curr_end_date.date()} para {target_end_date.date()}.")

    shift_value = (target_end_date - curr_end_date)
    df["Data da compra"] = (df["Data da compra"] + shift_value).dt.date
    df["Data de envio"] = (df["Data de envio"] + shift_value).dt.date
    return df

In [ ]:
def preprocess(df):
    print("\tRemovendo coluna Vendas")
    df = df.drop(columns="Vendas")
    print("\tRemovendo coluna Lucro")
    df["Lucro"] = df["Lucro"].abs()
    df = shift_dates_to_end_at(df, TARGET_END_DATE)
    
    assert df["Data da compra"].max() == TARGET_END_DATE
    return df

## Data selection functions

In [ ]:
def set_sent_til(df, date):
    """Marca como missing todas as Data de envio maiores do que date"""
    print(f"\tLimpando datas de envio maiores que {date}.")
    df = df.copy()
    df.loc[df["Data de envio"] > date, "Data de envio"] = np.nan
    return df


def get_data_til(df, date):
    """Seleciona todas as informações de compra existentes até a data date"""
    print(f"\tSelecionando base histórica até o dia {date}.")
    df = df[df["Data da compra"] <= date]
    return set_sent_til(df, date)

## Load to Sheets functions

In [ ]:
import os

from oauth2client.service_account import ServiceAccountCredentials
import gspread
from df2gspread import df2gspread as d2g


def get_credentials():
    """Se autentica para utilizar a API do Google Sheets"""
    scope = [
        'https://spreadsheets.google.com/feeds',
        'https://www.googleapis.com/auth/drive'
    ]

    credentials = ServiceAccountCredentials.from_json_keyfile_name(
        'superloja-bi-iti-ufscar.json', 
        scope
    )
    gspread.authorize(credentials)
    return credentials


def send_data_to_sheets(df, spreadsheet_key):
    """Envia a tabela para o Google Sheets especificado"""
    credentials = get_credentials()
    wks_name = 'Vendas'
    print("\tEnviando dados para Google Sheets")
    d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=False)

## Run pipeline

In [ ]:
# Extract
print("Extracting from original database")
df = pd.read_excel(FILE_PATH)
# Transform
print("Transforming data")
preprocessed_df = preprocess(df)
processed_df = get_data_til(preprocessed_df, TODAY)
# Load
print("Loading data to destination database")
send_data_to_sheets(processed_df, SPREADSHEET_KEY)